In [ ]:
#with this notebook, you can calculate aspect from dem data.

In [1]:
from osgeo import gdal,ogr
import pandas as pd
import geopandas
from shapely import wkt
from itertools import compress

In [2]:
def ListofExtensionAndName(directory,extension):
     
        if len(directory) != None:
            import os
            FilesList = []
            FileName=[]
            for root, subdirectory, files in os.walk(directory):
                for file in files:
                    if file.endswith(extension):
                        FilesList.append(os.path.join(root,file))
                        base=os.path.basename(file)
                        FileName.append(os.path.splitext(base)[0])

            return FilesList,FileName
        else:
            print("no"+ extension +"file for this directory")

In [3]:
def ImageBoundry(FilePath):
        #for aspect data
        imgname=FilePath

        from osgeo import gdal,ogr
        ds = gdal.Open(FilePath)
        gt = ds.GetGeoTransform()  # captures origin and pixel size

        ULC = gdal.ApplyGeoTransform(gt, 0, 0) #Upper Left Corner
        URC = gdal.ApplyGeoTransform(gt, ds.RasterXSize, 0) #Upper Right Corner
        LLC = gdal.ApplyGeoTransform(gt, 0, ds.RasterYSize) #Lower Left Corner
        LRC = gdal.ApplyGeoTransform(gt, ds.RasterXSize, ds.RasterYSize) #Lower Right Corner
                   
        ring = ogr.Geometry(ogr.wkbLinearRing)
        ring.AddPoint_2D(ULC[0], ULC[1])
        ring.AddPoint_2D(URC[0], URC[1])
        ring.AddPoint_2D(LRC[0], LRC[1])
        ring.AddPoint_2D(LLC[0], LLC[1])
        ring.AddPoint_2D(ULC[0], ULC[1]) 
        poly=ogr.Geometry(ogr.wkbPolygon)
        poly.AddGeometry(ring)
        poly.ExportToJson
        return poly.ExportToJson() ,imgname

In [ ]:
ImageList=list(ListofExtensionAndName(r"/data/dem_files",".tif"))


In [153]:
ImgboundaryList=[]

In [154]:
for i in ImageList[0]:
    imgboundary=ImageBoundry(i)
    gdf_new = geopandas.read_file(imgboundary[0])
    ImgboundaryList.append(i)
    gdf=gdf.append(gdf_new)


In [170]:
Mosaiclist=[]
imglist=[]

In [172]:
for i in ImageList[0]:
    imgindex=ImageList[0].index(i)
    mozaikname=ImageList[1][imgindex]
    
    imgboundary2=ImageBoundry(i)
    gdf_mosaic = geopandas.read_file(imgboundary2[0])
    gdfmosaic_geo=gdf_mosaic.geometry
    dist=gdfmosaic_geo.distance(gdf.geometry)
    for index,j in enumerate(dist,0):        
        if j <0.5:
           
            imglist.append(ImgboundaryList[index])
            
            
    Mosaiclist.append(imglist)
    mosaicpath="D:/Aspect_Mozaik"+ "/"+mozaikname+'.tif'
    gdal.Warp(mosaicpath,imglist)
    imglist=[]
    
    

In [106]:
aspect_result="data/Aspect_Result"


In [2]:
#below code, we create .bat file for command line

In [119]:
for index,i in enumerate(MozaikPath[0],0):
    outputimg=aspect_result+"/"+MozaikPath[1][index]+".tif"
    aspect_code= "gdaldem aspect"+ " "+ i + " "+ outputimg + " "+ "-of GTiff -b 1 -zero_for_flat -compute_edges"
    with open(r'data/aspect_calculation.bat', 'a') as the_file:
        the_file.write(aspect_code+"\n")

    